# Scraper

In [1]:
import bs4
import urllib.request

## Le Monde main page

Get the main page in order to get all the article links. 

In [2]:
lemonde = bs4.BeautifulSoup( urllib.request.urlopen("http://www.lemonde.fr/").read(), "lxml")

In [3]:
links = []
for a in lemonde('a'):
    # Avoid subscribed edition and take only article links
    if len(a('', {'class':'marqueur_restreint'})) == 0 and "/article/" in a['href']:
        links.append(a['href'])
len(links)

141

## Article example

Get an article and extract the useful information.

In [4]:
lemonde_article_url = "http://www.lemonde.fr" + links[0]
article_lemonde = bs4.BeautifulSoup( urllib.request.urlopen(lemonde_article_url).read(), "lxml" )
print( "Article: %s" % lemonde_article_url)
print( "Length: %d characters" % len(article_lemonde.text))

Article: http://www.lemonde.fr/paradise-papers/article/2017/11/06/au-fond-quel-est-le-probleme-avec-les-paradis-fiscaux_5210700_5209585.html
Length: 56485 characters


### Useful information

After a study of what is displayed on an article, I chose to extract those elements:
 - Title
 - Url
 - Article category
 - Writer
 - Article description
 - Publish Time
 - Update Time
 - Article content
 - Related article titles
 
Those elements are not loaded with the main static HTML. It is necessary to make supplementary requests or/and execute javascript.
In a first time, I won't scrape them. As, there is already enough to practice with the previous elements.
 - Comments
     - Writer
     - Date
     - Content
 - \# facebook shares

In [5]:
# Title
if len( article_lemonde('h1', {'class': 'tt2'}) ) > 0:
    title = article_lemonde('h1', {'class': 'tt2'})[0].string
    print ( "* Title: %s\n" % title)

# Simple scrap
if len( article_lemonde('', {'class':'description-article'}) ) > 0:
    article_description = article_lemonde('', {'class':'description-article'})[0].string
    print ( "* Description: \n%s\n" % article_description)

if len( article_lemonde('div', {'id': 'articleBody'}) ) > 0:
    article_content = article_lemonde('div', {'id': 'articleBody'})[0].text.strip()
    limit = len(article_content) if len(article_content)<500 else 500 
    print( "* Article content (cut): \n%s...\n" % article_content[0:500])

related_articles = []
if len( article_lemonde('aside', {'class':'bloc_base meme_sujet'}) ) > 0:
    for art in article_lemonde('aside', {'class':'bloc_base meme_sujet'})[0]('a'):
        related_articles.append(art.text)
    if len( related_articles ) > 0:
        print ( "* First related article title: \n%s" % related_articles[0].strip())

* Title: Au fond, quel est le problème avec les paradis fiscaux ?

* Description: 
Le débat sur les paradis fiscaux ne se limite pas à une simple concurrence entre des pays qui taxent beaucoup et d’autres qui taxent peu.

* Article content (cut): 
« Les paradis fiscaux, c’est terminé ! » Huit ans après l’annonce tonitruante de Nicolas Sarkozy, les « Paradise Papers » viennent d’apporter une nouvelle fois la preuve que le président français avait parlé un peu trop vite. Car si des progrès déterminants ont été accomplis ces dernières années au rythme des différents scandales financiers, le problème reste entier.
Mais finalement, quel est le problème et de quoi parle-t-on ? La question des paradis fiscaux recouvre en fait deux problématiques...

* First related article title: 
Les « Paradise Papers », nouvelles révélations sur les 350 milliards cachés de l’évasion fiscale


In [6]:
# Category
offset = len("http://www.lemonde.fr/")-1
slash1 = lemonde_article_url.find('/', offset)+1
slash2 = lemonde_article_url.find('/', slash1)+1
if ( slash1 >= 0 and slash2 >= 0):
    category_1 = lemonde_article_url[slash1:slash2-1]
    print ( "* Category 1: %s" % category_1)

# Writer
if len( article_lemonde('span', {'id': 'publisher'}) ) > 0:
    writer = article_lemonde('span', {'id': 'publisher'})[0].string
    print ( "* Writer: %s" % writer)

# Date
if len( article_lemonde('time', {'itemprop': 'datePublished'}) ) > 0:
    publish_time = article_lemonde('time', {'itemprop': 'datePublished'})[0]['datetime']
    print ( "* Publish time: %s" % publish_time)
if len( article_lemonde('time', {'itemprop': 'dateModified'}) ) > 0:
    update_time = article_lemonde('time', {'itemprop': 'dateModified'})[0]['datetime']
    print ( "* Update time: %s" % update_time)

* Category 1: paradise-papers
* Writer: Le Monde
* Publish time: 2017-11-06T09:11:48+01:00


### Advanced scrap [Just to see]

Scrap inside of an article content and try to organize it in order to have more information like different titles and paragraphs. This method is more complex and less robust as the HTML references can easily change from article to article.

In a first place, I will only focus on a simple scrap for the ranking system and don't use this one. 

The code below might not return all the information as the HTML references are not enough general for all articles.

In [7]:
# Complete

if len( article_lemonde('h2', {'class':'taille_courante'}) ) > 0:
    article_summary = article_lemonde('h2', {'class':'taille_courante'})[0].text
    print ( "* Summary: \n%s" % article_summary)


print()
article_paragraphs = []
for paragraph in article_lemonde('div', {'id':'articleBody'})[0]('p'):
    if paragraph.get('class') is None:
        article_paragraphs.append(paragraph.text)
print ( "* First paragraph: \n%s" % article_paragraphs[0])

article_intertitres = []
for inter in article_lemonde('h2', {'class':'intertitre'}):
    article_intertitres.append(inter.text)
if len( article_intertitres ) > 0:
    print ( "* First inter title: \n%s" % article_intertitres[0])

* Summary: 
« Les paradis fiscaux, c’est terminé ! » Huit ans après l’annonce tonitruante de Nicolas Sarkozy, les « Paradise Papers » viennent d’apporter une nouvelle fois la preuve que le président français avait parlé un peu trop vite. Car si des progrès déterminants ont été accomplis ces dernières années au rythme des différents scandales financiers, le problème reste entier.

* First paragraph: 
Mais finalement, quel est le problème et de quoi parle-t-on ? La question des paradis fiscaux recouvre en fait deux problématiques : les impôts et l’opacité financière.
* First inter title: 
1. Les impôts
